In [20]:
import random
import numpy as np

In [21]:
AGENT_HOME     = 0
AGENT_ANYWHERE = 1
AGENT_ORE      = 2
AGENT_RADAR    = 3
AGENT_BURY     = 4
#AGENT_TRAP     = 4

ACTION_WAIT           = 0
ACTION_DIG_ORE        = 1
ACTION_DIG_SURELY_ORE = 2
ACTION_DIG_BIG_RADAR  = 3
ACTION_DIG_RADAR      = 4
ACTION_REQUEST_RADAR  = 5
ACTION_MOVE_HOME      = 6
#ACTION_NONE           = 7

class Agent:
    
    def __init__(self):
        self.state = AGENT_HOME
        self.action = ACTION_WAIT

class AgentState:
    
    def __init__(self):
        self.agent = []
        self.size  = 0
        self.start = 0
        self.index = 0 
        
        
    def append(self, Agent):
        self.agent.append(Agent)
        self.size = self.size + 1
        
    def pop(self):
        self.agent.pop()
        self.size = self.size - 1
        self.start = self.start + 1
    
    def __getitem__(self, index):
        return self.agent[index]
        
class Action:
    
    def available_functions(self):
        if self.agent_state[-1].state == AGENT_HOME :
            t_tuple = [-1] * 7
            i = 0
            for func_in_list in self.is_function_home :
                if func_in_list is not None:
                    t_tuple[i] = -1 if func_in_list() == None else 1
                i = i + 1
            t_tuple = tuple(t_tuple)
            
        elif self.agent_state[-1].state == AGENT_RADAR :
            t_tuple = [-1] * 7
            i = 0
            for func_in_list in self.is_function_radar :
                if func_in_list is not None:
                    t_tuple[i] = -1 if func_in_list() == None else 1
                i = i + 1
            t_tuple = tuple(t_tuple)

        elif self.agent_state[-1].state == AGENT_ANYWHERE :
            t_tuple = [-1] * 7
            if self.agent_state[-1].action == ACTION_WAIT :
                i = 0
                for func_in_list in self.is_function_anywhere :
                    if func_in_list is not None:
                        t_tuple[i] = -1 if func_in_list() == None else 1
                    i = i + 1
                t_tuple = tuple(t_tuple)
            else :
                t_tuple[self.agent_state[-1].action] = 1
                t_tuple = tuple(t_tuple)
            
        elif self.agent_state[-1].state == AGENT_ORE :
            t_tuple = [-1] * 7
            t_tuple[ACTION_MOVE_HOME] = 1
            t_tuple = tuple(t_tuple)
            
        elif self.agent_state[-1].state == AGENT_BURY :
            t_tuple = [-1] * 7
            t_tuple[ACTION_WAIT] = 1
            t_tuple = tuple(t_tuple)
        
        return t_tuple
    
    def call_functions(self, index):
        if self.agent_state[-1].state == AGENT_HOME :
            self.list_function_home[index]()
        
        elif self.agent_state[-1].state == AGENT_ANYWHERE :
            self.list_function_anywhere[index]()
        
        elif self.agent_state[-1].state == AGENT_ORE :
            self.list_function_ore[index]()
            
        elif self.agent_state[-1].state == AGENT_RADAR :
            self.list_function_radar[index]()
        
        elif self.agent_state[-1].state == AGENT_BURY :
            self.list_function_bury[index]()
        
    def isHome(self):
        return 1
    
    def isOre(self):
        result = random.randint(1,20)
        current_agent = self.agent_state[-1]
        if result >= self.hazardous :
            if current_agent.state == AGENT_HOME :
                self.distance_ore = random.randint(0, 20)
            if current_agent.state == AGENT_ANYWHERE :
                self.distance_ore = random.randint(0, 16)
            return (self.distance_ore)
        return None

    def isSurelyOre(self):
        result = random.randint(1,20)
        current_agent = self.agent_state[-1]
        if result >= self.hazardous :
            if current_agent.state == AGENT_HOME :
                self.distance_surely_ore = random.randint(0, 20)
            if current_agent.state == AGENT_ANYWHERE :
                self.distance_surely_ore = random.randint(0, 16)
            return (self.distance_surely_ore)
        return None

    def isBigRadar(self):
        if self.distance_big_radar <= 0 :
            self.distance_big_radar = random.randint(-1,11)

        if self.distance_big_radar == -1:
            return None
        return (self.distance_big_radar)

    def isRadar(self):
        if self.distance_radar <= 0 :
            self.distance_radar = random.randint(-1,7)

        if self.distance_radar == -1:
            return None
        return (self.distance_radar)

    def isRequestRadar(self):
        return 1

    def waiting(self):
        current_agent = self.agent_state[-1]
        self.agent_state.append(current_agent)
        self.agent_state.pop()
        return None
    
    def digOre(self):
        current_agent = self.agent_state[-1]
        if self.distance_ore <= 1 :
            result = random.randint(-2,20)
            if result <= -1 :
                current_agent.state = AGENT_BURY
            else :
                current_agent.state = AGENT_ORE
            current_agent.action = ACTION_WAIT
        
        else :
            self.distance_ore = self.distance_ore - 4
            self.distance_ore = self.distance_ore if self.distance_ore > 0 else 0
            self.action = ACTION_DIG_ORE if self.distance_ore > 0 else ACTION_WAIT
        
        self.agent_state.append(current_agent)
        self.agent_state.pop()
        return None

    def digSurelyOre(self):
        current_agent = self.agent_state[-1]
        if self.distance_surely_ore <= 1 :
            result = random.randint(-4,20)
            if result <= -1 :
                current_agent.state = AGENT_BURY
            else :
                current_agent.state = AGENT_ORE
            current_agent.action = ACTION_WAIT
        
        else :
            self.distance_surely_ore = self.distance_surely_ore - 4
            self.distance_surely_ore = self.distance_surely_ore if self.distance_surely_ore > 0 else 0
            self.action = ACTION_DIG_SURELY_ORE if self.distance_surely_ore > 0 else ACTION_WAIT
        
        self.agent_state.append(current_agent)
        self.agent_state.pop()
        return None
        
    def digBigRadar(self):
        current_agent = self.agent_state[-1]
        if self.distance_big_radar <= 1 :
            result = random.randint(-2,20)
            if result <= -1 :
                current_agent.state = AGENT_BURY
            elif result == 0 :
                current_agent.state = AGENT_ORE
                self.hazardous = self.hazardous - 2
                self.hazardous = self.hazardous if self.hazardous > self.min_hazardous else self.min_hazardous
            else:
                current_agent.state = AGENT_ANYWHERE
                self.hazardous = self.hazardous - 2
                self.hazardous = self.hazardous if self.hazardous > self.min_hazardous else self.min_hazardous
            current_agent.action = ACTION_WAIT
        
        else :
            self.distance_big_radar = self.distance_big_radar - 4
            self.distance_big_radar = self.distance_big_radar if self.distance_big_radar > 0 else 0
            self.action = ACTION_DIG_BIG_RADAR if self.distance_big_radar > 0 else ACTION_WAIT
        
        self.agent_state.append(current_agent)
        self.agent_state.pop()
        return None
        
    def digRadar(self):
        current_agent = self.agent_state[-1]
        if self.distance_big_radar <= 1 :
            result = random.randint(-2,20)
            if result <= -1 :
                current_agent.state = AGENT_BURY
            elif result == 0 :
                current_agent.state = AGENT_ORE
                self.hazardous = self.hazardous - 1
                self.hazardous = self.hazardous if self.hazardous > self.min_hazardous else self.min_hazardous
            else:
                current_agent.state = AGENT_ANYWHERE
                self.hazardous = self.hazardous - 1
                self.hazardous = self.hazardous if self.hazardous > self.min_hazardous else self.min_hazardous
            current_agent.action = ACTION_WAIT
        
        else :
            self.distance_big_radar = self.distance_big_radar - 4
            self.distance_big_radar = self.distance_big_radar if self.distance_big_radar > 0 else 0
            self.action = ACTION_DIG_RADAR if self.distance_big_radar > 0 else ACTION_WAIT
        
        self.agent_state.append(current_agent)
        self.agent_state.pop()
        return None
        
    def requestRadar(self):
        current_agent = self.agent_state[-1]
        current_agent.state = AGENT_RADAR
        self.agent_state.append(current_agent)
        self.agent_state.pop()
        return None
        
    def goHome(self):
        current_agent = self.agent_state[-1]
        if current_agent.state == AGENT_ORE :
            self.score = self.score + 1
        current_agent.state = AGENT_HOME
        self.agent_state.append(current_agent)
        self.agent_state.pop()
        return None
        
    def __init__(self):
        self.is_okay = True
        self.hazardous = 20
        self.min_hazardous = 5
        
        self.distance_ore = -1
        self.distance_surely_ore = -1
        self.distance_radar = -1
        self.distance_big_radar = -1
        
        self.list_function_home     = (None   , self.digOre, self.digSurelyOre, None       , None    , self.requestRadar, None  )
        self.list_function_anywhere = (None   , self.digOre, self.digSurelyOre, None       , None    , self.requestRadar, None )
        self.list_function_ore      = (None   , None  , None        , None       , None    , None        , self.goHome)
        self.list_function_radar    = (None   , self.digOre, self.digSurelyOre, self.digBigRadar, self.digRadar, None        , None  )
        self.list_function_bury     = (self.waiting, None  , None        , None       , None    , None        , None  )
        
        self.is_function_home       = (None, self.isOre, self.isSurelyOre, None      , None   , self.isRequestRadar, None )
        self.is_function_anywhere   = (None, self.isOre, self.isSurelyOre, None      , None   , self.isRequestRadar, None )
        self.is_function_radar      = (None, self.isOre, self.isSurelyOre, self.isBigRadar, self.isRadar, None          , None )
        
        self.agent_state = AgentState()
        self.score = 0


Q Learning simple program

From this site

https://www.linkedin.com/pulse/simple-q-learning-algorithm-python-surya-kari

In [22]:
environment = [(0,0,1),(1,0,2),(2,0,5),(3,1,1),(4,1,2),(5,1,5),(6,2,6),(7,3,1),(8,3,2),(9,3,3),(10,3,4),(11,4,0)]

states = 5
actions = 7
goal = 6

## R reward matrix
R = np.matrix(np.ones(shape=[states,actions]))
R = R * -1

for (i,x,y) in environment:
    R[x,y] = 100 if i == goal else 0

## Q quality matrix : Our brain
Q = np.matrix(np.zeros(shape=[states,actions]))

print('R \n{}'.format(R))
print('Q \n{}'.format(Q))

R 
[[ -1.   0.   0.  -1.  -1.   0.  -1.]
 [ -1.   0.   0.  -1.  -1.   0.  -1.]
 [ -1.  -1.  -1.  -1.  -1.  -1. 100.]
 [ -1.   0.   0.   0.   0.  -1.  -1.]
 [  0.  -1.  -1.  -1.  -1.  -1.  -1.]]
Q 
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [23]:
action = Action()
agent = Agent()

agent_state = action.agent_state
agent_state.append(agent)


In [24]:
def one_loop():
    previous_state = agent_state[-1].state
    
    result_tuple = action.available_functions()
    result = np.array([[i for i in result_tuple]])

    available_actions = np.where(result >= 0)[1]
    choose_action = random.choice(available_actions)

    action.call_functions(choose_action)
    
    current_state = agent_state[-1].state
    return (previous_state, current_state, choose_action)
    
gamma = 0.8
def Q_update(previous_state,current_state,action,gamma):
    # First the max Q part
    Max_Q = np.max(Q[current_state,])
    Q[previous_state,action] = R[previous_state,action] + gamma * Max_Q


In [25]:
#(prev_state, curr_state, choo_action) = one_loop()
#Q_update(prev_state,curr_state,choo_action,gamma)
print('Q\n {}'.format(Q))

Q
 [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [26]:
for i in range(0,1000):
    (prev_state, curr_state, choo_action) = one_loop()
    Q_update(prev_state,curr_state,choo_action,gamma)

print('Q\n {}'.format(np.around(Q,3)))

Q
 [[  0.     90.836  51.2     0.      0.    118.78    0.   ]
 [  0.     90.836  90.836   0.      0.    118.78    0.   ]
 [  0.      0.      0.      0.      0.      0.    190.836]
 [  0.    141.932 148.475   0.     54.78    0.      0.   ]
 [  0.      0.      0.      0.      0.      0.      0.   ]]
